# LLSQ Fit
Use background corrected reference spectra to fit an unknown. 

In [ ]:
using NeXLSpectrum
using DataFrames, Gadfly, CSV
using LinearAlgebra

Load the unknown spectrum and the reference spectra.  Plot them.

In [ ]:
path = joinpath(@__DIR__,"K309")
unk = loadspectrum(joinpath(path, "K309.msa"))
unk[:Composition]=parse(Material, "0.3872*O+0.0794*Al+0.1870*Si+0.1072*Ca+0.1049*Fe+0.1343*Ba", name="K309")
unk[:Coating]=Film(parse(Material,"C",density=1.9),10.0e-7)
stds = map(("Al2O3", "BaF2", "CaF2", "Fe", "Si", "Ti" )) do fn 
  s = loadspectrum(joinpath(path,"$fn std.msa"))
  s[:Composition]=parse(Material,fn)
  s[:Coating]=Film(parse(Material,"C",density=1.9),10.0e-7)
  s
end
elems = collect(elms(unk, true))
set_default_plot_size(8inch,4inch)
plot(unk,stds...,klms=elems,xmax=8.0e3)

In [ ]:
det = matching(unk,132.0, 110)
resp = detectorresponse(det,SDDEfficiency(ModeledWindow(MoxtekAP33())))

unkb = fittedcontinuum(unk, det, resp)
stdsb = map(s->fittedcontinuum(s, det, resp), stds);

In [ ]:
plot(Iterators.flatten(zip(stdsb, stds))..., xmax=8.0e3, yscale=0.008)

In [ ]:
plot(unkb, unk, autoklms=true,xmax=8.0e3, yscale=0.04) 

In [ ]:
unkc = subtractcontinuum(unk, det, resp)
stdsc = map(s->subtractcontinuum(s, det, resp), stds)
plot(unkc, stdsc..., autoklms=true, xmax=8.0e3, yscale=0.01)

In [ ]:
plot(stdsc[4], autoklms=true, xmax=8.0e3, yscale=0.01)

Seems to work - at least the basic concept.

In [ ]:
function fit_spectrum(
    spec::Spectrum, 
    refs::Dict{Element, <:Spectrum}, 
    det::EDSDetector,
    resp::AbstractArray{<:Real,2};
    brem::Type{<:NeXLBremsstrahlung} = Castellano2004a,
    mc::Type{<:MatrixCorrection} = Riveros1993,
 )# ::Vector{KRatio}
  cuds = []
  for (elm, ref) in refs
    cmod = ContinuumModel(
         ref[:Composition],
         ref[:BeamEnergy],
         ref[:TakeOffAngle],
         matrixcorrection = mc,
         bremsstrahlung = brem)
    model = resp * map(e -> e > 50.0 ? emitted(cmod, e) : 0.0, energyscale(spec))
    cud = map(NeXLSpectrum.labeledextents(characteristic(elm, alltransitions, 0.001), det, 0.001)) do le
       cxrs, ur = le 
       ( cxrs, ur, ref[ur]-model[ur], ref )
    end
    append!(cuds, cud)  
  end
  ascontiguous = let
    scuds = sort(cuds, lt=(c1,c2)->first(c1[2])<first(c2[2]))
    res = [ scuds[1][2] ]
    for (_, roi, _, _) in scuds[2:end]
      if length(intersect(res[end], roi)) > 0
        res[end] = min(first(roi), first(res[end])):max(last(roi), last(res[end]))
      else
        push!(res, roi)
      end
    end
    res
  end
  cmod = ContinuumModel(
         unk[:Composition],
         unk[:BeamEnergy],
         unk[:TakeOffAngle],
         matrixcorrection = mc,
         bremsstrahlung = brem)
  model = resp * map(e -> e > 50.0 ? emitted(cmod, e) : 0.0, energyscale(spec))
  y = counts(unk) - model
  res = UncertainValues[]
  for roi in ascontiguous
    fcuds = filter(cud->length(intersect(cud[2], roi))>0, cuds)
    a = zeros(Float64, (length(roi), length(fcuds)))
    labels, scales = Label[], Float64[]
    for (i, cud) in enumerate(fcuds)
      (cxrs, croi, cdata, cref) = cud
      a[first(croi)-first(roi)+1:last(croi)-first(roi)+1,i] = cdata
      push!(labels, CharXRayLabel(spec, croi, cxrs))
      push!(scales, dose(cref) / dose(unk))
    end
    push!(res, diagm(scales)*NeXLSpectrum.olspinv(y[roi], a, 1.0, labels))
  end
  return cat(res...)
end

In [ ]:
refs = Dict(n"O"=>stds[1],n"Ba"=>stds[2],n"Ca"=>stds[3],n"Fe"=>stds[4],n"Si"=>stds[5],n"Ti"=>stds[6])
fit_spectrum(unk, refs, det, resp)

Let's compare this to the filter-fit result.

In [ ]:
ffrs = references( [
        reference(n"O", stds[1]),
        reference(n"Ba", stds[2]),
        reference(n"Ca", stds[3]),
        reference(n"Fe", stds[4]),
        reference(n"Si", stds[5]),
        reference(n"Ti", stds[6])
    ], det)
ffres=NeXLSpectrum.fit_spectrum(unk, ffrs)
asa(DataFrame, ffres)

Clearly, there is some similarity between the numbers but the agreement is not what I'd like.

Who do you trust?  Well, I trust the filter-fit results.  I'm very confident that the filter-fit algorithm produces accurate k-ratios.  Less so for the background corrected fit.

NWMR